## Hazard indicators
Physrisk is primarily designed to run 'bottom-up' calculations that model the impact of climate hazards on large numbers of individual assets (including natural) and operations. These calculations can be used to assess financial risks or socio-economic impacts. To do this physrisk collects:

- hazard indicators and
- models of vulnerability of assets/operations to hazards.

Hazard indicators – that is, quantities that provide the information about hazards needed by the vulnerability models – are collected from a variety of sources. OS-Climate consolidates public domain hazard indicators and also provides the means to combine these with commercial data.

We start with public domain indicators: which ones are available and how can these be obtained?

#### Requesting hazard indicators via the physrisk API

Using physrisk, hazard indicators can be obtained directly from source, as long as the user has the necessary API keys. Alternatively, the API – which is simply a hosted instance of physrisk – can be used, albeit the number of requests will be restricted to 30,000 latitudes and longitudes.

We give a walk-through of the hazard indicator API below, as well as examples of how to obtain the inventory of available hazard indicators.

In [1]:
# UNCOMMENT TO RUN THIS NOTEBOOK
# pip install nbformat pandas plotly requests

# UNCOMMENT TO RUN THE SECOND PART OF THIS NOTEBOOK:
# pip install physrisk-lib

import pandas as pd
import plotly.graph_objs as go
import pprint as pp
import requests

from dotenv import load_dotenv
from IPython.display import Markdown, display
from physrisk.container import Container
from plotly.subplots import make_subplots
import plotly.io

plotly.io.renderers.default = "notebook"

In [2]:
base_url = "https://physrisk-api-sandbox.apps.odh-cl1.apps.os-climate.org/api/"

request = {
    "items": [
        {
            "longitudes": [69.4787, 68.71, 20.1047, 19.8936, 19.6359, 0.5407, 6.9366, 6.935, 13.7319, 13.7319],
            "latitudes": [34.556, 35.9416, 39.9116, 41.6796, 42.0137, 35.7835, 36.8789, 36.88, -12.4706, -12.4706],
            "request_item_id": "my_flood_request",
            "hazard_type": "RiverineInundation",
            "indicator_id": "flood_depth",
            "scenario": "historical",
            "year": 1980,
        },
        {
            "longitudes": [69.4787, 68.71, 20.1047, 19.8936, 19.6359, 0.5407, 6.9366, 6.935, 13.7319, 13.7319],
            "latitudes": [34.556, 35.9416, 39.9116, 41.6796, 42.0137, 35.7835, 36.8789, 36.88, -12.4706, -12.4706],
            "request_item_id": "my_flood_request",
            "hazard_type": "RiverineInundation",
            "indicator_id": "flood_depth",
            "scenario": "rcp8p5",
            "indicator_model_gcm": "NorESM1-M",  # optional: can specify
            "year": 2050,
        },
        {
            "longitudes": [114.089],
            "latitudes": [22.4781],
            "request_item_id": "my_wind_request_ssp585",
            "hazard_type": "Wind",
            "indicator_id": "max_speed",
            "scenario": "historical",
            "path": "wind/iris/v1/max_speed_{scenario}_{year}",
            # if path is specified then that particular data array is used
            "year": 2010,
        },
        {
            "longitudes": [114.089],
            "latitudes": [22.4781],
            "request_item_id": "my_wind_request_histo",
            "hazard_type": "Wind",
            "indicator_id": "max_speed",
            "scenario": "ssp585",
            "path": "wind/iris/v1/max_speed_{scenario}_{year}",
            "year": 2050,
        },
        {
            "longitudes": [114.089],
            "latitudes": [22.4781],
            "request_item_id": "my_fire_request",
            "hazard_type": "Fire",
            "indicator_id": "fire_probability",
            "scenario": "ssp585",
            "path": "fire/jupiter/v1/fire_probability_{scenario}_{year}",
            "year": 2040,
        },
    ]
}
url = base_url + "get_hazard_data"
response = requests.post(url, json=request).json()
flood_results_baseline, flood_results_rcp585 = (
    response["items"][0]["intensity_curve_set"],
    response["items"][1]["intensity_curve_set"],
)
wind_results_baseline, wind_results_ssp585 = (
    response["items"][2]["intensity_curve_set"],
    response["items"][3]["intensity_curve_set"],
)
fire_results = response["items"][4]["intensity_curve_set"]

In [10]:
fig1 = make_subplots(rows=1, cols=2)
fig1.add_scatter(x=flood_results_baseline[0]["index_values"], y=flood_results_baseline[0]["intensities"], name="baseline flood", row=1, col=1)
fig1.add_scatter(x=flood_results_rcp585[0]["index_values"], y=flood_results_rcp585[0]["intensities"], name="flood RCP 8.5 2050", row=1, col=1)
fig1.update_xaxes(title="Return period (years)", title_font={"size": 14}, row=1, col=1, type="log")
fig1.update_yaxes(title="Flood depth (m)", title_font={"size": 14}, row=1, col=1)
fig1.add_scatter(x=wind_results_baseline[0]["index_values"], y=wind_results_baseline[0]["intensities"], name="baseline wind", row=1, col=2)
fig1.add_scatter(x=wind_results_ssp585[0]["index_values"], y=wind_results_ssp585[0]["intensities"], name="wind SSP585 2050", row=1, col=2)
fig1.update_xaxes(title="Return period (years)", title_font={"size": 14}, row=1, col=2, type="log")
fig1.update_yaxes(title="Max (1 minute) wind speed (m/s)", title_font={"size": 14}, row=1, col=2)

### Accessing the inventory of hazard indicators

Hazard indicator data are 3 dimensional arrays, the dimensions being ('index', 'spatial y', 'spatial x'). In the case of a EPSG:4326 co-ordinate reference system (CRS), for example, this would be ('index', 'latitude', 'longitude'). The data can be in any (CRS); generally the CRS of the original CRS is retained. API requests specify location via latitude and longitude and physrisk performs the necessary conversion.

The 'index' depends on the nature of the hazards. For acute hazards – potential occurrence of events such as flood or tropical cyclone – 'index' will be the return period of the event. For chronic hazards its definition can vary. Chronic hazard indicators may be threshold-based, for example 'mean degree days per year above threshold' or 'mean days per year above temperature threshold'; in these cases 'index' gives the thresholds. Chronic hazards may comprise just a single value (e.g. annual probability of fire) in which case there is just a single element in 'index'.    

The inventory is a list of 'hazard resources' and each resource corresponds to a single array. This means that there are separate entries for the same hazard indicators inferred from different (e.g. CMIP6 or CORDEX) climate models.

For the requests above, the 'path' of the array can be specified explicitly. This is the unique identifier for the hazard resource. But path can also be omitted in which can physrisk applies a set of rules to select a particular array. 

In [4]:
url = base_url + "get_hazard_data_availability"
hazard_resources = requests.post(url, json=request).json()["models"]

In [5]:
def printmd(string):
    display(Markdown(string))


hazard_types = set(r["hazard_type"] for r in hazard_resources)
print(f"Hazards: {hazard_types}")
wind_resources = [r for r in hazard_resources if r["hazard_type"] == "Wind"]
print(f"{len(hazard_resources)} hazard resources in the inventory, of which {len(wind_resources)} are Wind resources.")
iris_model_resource = next(r for r in wind_resources if "iris" in r["path"])
# pp.pprint(iris_model_resource[0])
path = iris_model_resource["path"]
print(f"The resource 'path' is a unique identifier. For the IRIS Wind resource, for example, this is: '{path}'.")
print(
    "Where resources are multi-dimensional arrays (as opposed to an external API), this is also the path of the array."
)
print("The available climate scenarios and years for the IRIS resource are:")
pp.pprint(iris_model_resource["scenarios"])
print("A description is provided in markdown:")
printmd(iris_model_resource["description"])
print("The whole resource:")
pp.pprint(iris_model_resource)

Hazards: {'ChronicHeat', 'Precipitation', 'Drought', 'Wind', 'RiverineInundation', 'WaterRisk', 'Subsidence', 'Fire', 'CombinedInundation', 'Hail', 'CoastalInundation'}
122 hazard resources in the inventory, of which 2 are Wind resources.
The resource 'path' is a unique identifier. For the IRIS Wind resource, for example, this is: 'wind/iris/v1/max_speed_{scenario}_{year}'.
Where resources are multi-dimensional arrays (as opposed to an external API), this is also the path of the array.
The available climate scenarios and years for the IRIS resource are:
[{'id': 'historical', 'years': [2010]},
 {'id': 'ssp119', 'years': [2050]},
 {'id': 'ssp245', 'years': [2050]},
 {'id': 'ssp585', 'years': [2050]}]
A description is provided in markdown:


Sparks, N., Toumi, R. The Imperial College Storm Model (IRIS) Dataset. *Sci Data* **11**, 424 (2024). <https://doi.org/10.1038/s41597-024-03250-y>
## The Imperial College Storm Model (IRIS) Dataset - Scientific Data
Assessing tropical cyclone risk on a global scale given the infrequency of landfalling tropical cyclones and the short period of reliable observations remains a challenge. Synthetic tropical cyclone datasets can help overcome these problems. Here we present a new global dataset created by IRIS, the ImpeRIal college Storm Model. IRIS is novel because, unlike other synthetic TC models, it only simulates the decay from the point of lifetime maximum intensity. This minimises the bias in the dataset. It takes input from 42 years of observed tropical cyclones and creates a 10,000 year synthetic dataset which is then validated against the observations. IRIS captures important statistical characteristics of the observed data. The return periods of the landfall maximum wind speed (1 minute sustained in m/s) are realistic globally. Climate model projections are used to adjust the life-time maximum intensity.

***Disclaimer***: There have been many improvements on the dataset. Contact Professor Toumi from the Imperial College London for improved data.

The whole resource:
{'description': 'Sparks, N., Toumi, R. The Imperial College Storm Model (IRIS) '
                'Dataset. *Sci Data* **11**, 424 (2024). '
                '<https://doi.org/10.1038/s41597-024-03250-y>\n'
                '## The Imperial College Storm Model (IRIS) Dataset - '
                'Scientific Data\n'
                'Assessing tropical cyclone risk on a global scale given the '
                'infrequency of landfalling tropical cyclones and the short '
                'period of reliable observations remains a challenge. '
                'Synthetic tropical cyclone datasets can help overcome these '
                'problems. Here we present a new global dataset created by '
                'IRIS, the ImpeRIal college Storm Model. IRIS is novel '
                'because, unlike other synthetic TC models, it only simulates '
                'the decay from the point of lifetime maximum intensity. This '
                'minimises the bias in the datase

## Writing hazard resources to a summary table

The code below creates a summary table for all hazard resources.

In [37]:
#from typing import List
import pandas as pd
import json

rcp_display_names = {"rcp2p6": "RCP 2.6", "rcp4p5": "RCP 4.5" ,"rcp6p0": "RCP 6.0", "rcp8p5": "RCP 8.5", }

def scenario_code(id: str):
    if id.startswith("ssp"):
        return id.upper()
    elif id.startswith("rcp"):
        return id.replace("rcp", "RCP ").replace("p", ".")
    else:
        return id

def format_scenario(scenarios):
    return ", ".join(f"{scenario_code(scen['id'])} ({', '.join(str(y) for y in scen['years'])})" for scen in scenarios)

resource_df = pd.json_normalize(hazard_resources)
resource_df = resource_df.filter(["hazard_type", "indicator_id", "display_name", "path", "scenarios"])
resource_df = resource_df.sort_values(by=["hazard_type", "indicator_id", "display_name"])
resource_df.scenarios = resource_df.scenarios.apply(lambda s: format_scenario(s))
resource_df = resource_df.rename(columns={"hazard_type": "Hazard type",
                                          "indicator_id": "Indicator ID",
                                          "display_name": "Display name",
                                          "path": "Path (ID)",
                                          "scenarios": "Scenarios",
                                          })

md = resource_df.to_markdown(index=False)
display(Markdown(md))

| Hazard type        | Indicator ID                 | Display name                                                             | Path (ID)                                                                         | Scenarios                                                                                                                                                                                                                                                 |
|:-------------------|:-----------------------------|:-------------------------------------------------------------------------|:----------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| ChronicHeat        | days/above/35c               | Days per year above 35°C (Jupiter)                                       | chronic_heat/jupiter/v1/days_above_35c_{scenario}_{year}                          | SSP126 (2020, 2030, 2040, 2050, 2075, 2100), SSP585 (2020, 2030, 2040, 2050, 2075, 2100)                                                                                                                                                                  |
| ChronicHeat        | days_tas/above/25c           | Days with average temperature above 25°C/ACCESS-CM2                      | chronic_heat/osc/v2/days_tas_above_25c_ACCESS-CM2_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/25c           | Days with average temperature above 25°C/CMCC-ESM2                       | chronic_heat/osc/v2/days_tas_above_25c_CMCC-ESM2_{scenario}_{year}                | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/25c           | Days with average temperature above 25°C/CNRM-CM6-1                      | chronic_heat/osc/v2/days_tas_above_25c_CNRM-CM6-1_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/25c           | Days with average temperature above 25°C/MIROC6                          | chronic_heat/osc/v2/days_tas_above_25c_MIROC6_{scenario}_{year}                   | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/25c           | Days with average temperature above 25°C/MPI-ESM1-2-LR                   | chronic_heat/osc/v2/days_tas_above_25c_MPI-ESM1-2-LR_{scenario}_{year}            | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/25c           | Days with average temperature above 25°C/NorESM2-MM                      | chronic_heat/osc/v2/days_tas_above_25c_NorESM2-MM_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/30c           | Days with average temperature above 30°C/ACCESS-CM2                      | chronic_heat/osc/v2/days_tas_above_30c_ACCESS-CM2_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/30c           | Days with average temperature above 30°C/CMCC-ESM2                       | chronic_heat/osc/v2/days_tas_above_30c_CMCC-ESM2_{scenario}_{year}                | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/30c           | Days with average temperature above 30°C/CNRM-CM6-1                      | chronic_heat/osc/v2/days_tas_above_30c_CNRM-CM6-1_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/30c           | Days with average temperature above 30°C/MIROC6                          | chronic_heat/osc/v2/days_tas_above_30c_MIROC6_{scenario}_{year}                   | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/30c           | Days with average temperature above 30°C/MPI-ESM1-2-LR                   | chronic_heat/osc/v2/days_tas_above_30c_MPI-ESM1-2-LR_{scenario}_{year}            | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/30c           | Days with average temperature above 30°C/NorESM2-MM                      | chronic_heat/osc/v2/days_tas_above_30c_NorESM2-MM_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/35c           | Days with average temperature above 35°C/ACCESS-CM2                      | chronic_heat/osc/v2/days_tas_above_35c_ACCESS-CM2_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/35c           | Days with average temperature above 35°C/CMCC-ESM2                       | chronic_heat/osc/v2/days_tas_above_35c_CMCC-ESM2_{scenario}_{year}                | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/35c           | Days with average temperature above 35°C/CNRM-CM6-1                      | chronic_heat/osc/v2/days_tas_above_35c_CNRM-CM6-1_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/35c           | Days with average temperature above 35°C/MIROC6                          | chronic_heat/osc/v2/days_tas_above_35c_MIROC6_{scenario}_{year}                   | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/35c           | Days with average temperature above 35°C/MPI-ESM1-2-LR                   | chronic_heat/osc/v2/days_tas_above_35c_MPI-ESM1-2-LR_{scenario}_{year}            | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/35c           | Days with average temperature above 35°C/NorESM2-MM                      | chronic_heat/osc/v2/days_tas_above_35c_NorESM2-MM_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/40c           | Days with average temperature above 40°C/ACCESS-CM2                      | chronic_heat/osc/v2/days_tas_above_40c_ACCESS-CM2_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/40c           | Days with average temperature above 40°C/CMCC-ESM2                       | chronic_heat/osc/v2/days_tas_above_40c_CMCC-ESM2_{scenario}_{year}                | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/40c           | Days with average temperature above 40°C/CNRM-CM6-1                      | chronic_heat/osc/v2/days_tas_above_40c_CNRM-CM6-1_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/40c           | Days with average temperature above 40°C/MIROC6                          | chronic_heat/osc/v2/days_tas_above_40c_MIROC6_{scenario}_{year}                   | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/40c           | Days with average temperature above 40°C/MPI-ESM1-2-LR                   | chronic_heat/osc/v2/days_tas_above_40c_MPI-ESM1-2-LR_{scenario}_{year}            | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/40c           | Days with average temperature above 40°C/NorESM2-MM                      | chronic_heat/osc/v2/days_tas_above_40c_NorESM2-MM_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/45c           | Days with average temperature above 45°C/ACCESS-CM2                      | chronic_heat/osc/v2/days_tas_above_45c_ACCESS-CM2_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/45c           | Days with average temperature above 45°C/CMCC-ESM2                       | chronic_heat/osc/v2/days_tas_above_45c_CMCC-ESM2_{scenario}_{year}                | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/45c           | Days with average temperature above 45°C/CNRM-CM6-1                      | chronic_heat/osc/v2/days_tas_above_45c_CNRM-CM6-1_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/45c           | Days with average temperature above 45°C/MIROC6                          | chronic_heat/osc/v2/days_tas_above_45c_MIROC6_{scenario}_{year}                   | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/45c           | Days with average temperature above 45°C/MPI-ESM1-2-LR                   | chronic_heat/osc/v2/days_tas_above_45c_MPI-ESM1-2-LR_{scenario}_{year}            | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/45c           | Days with average temperature above 45°C/NorESM2-MM                      | chronic_heat/osc/v2/days_tas_above_45c_NorESM2-MM_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/50c           | Days with average temperature above 50°C/ACCESS-CM2                      | chronic_heat/osc/v2/days_tas_above_50c_ACCESS-CM2_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/50c           | Days with average temperature above 50°C/CMCC-ESM2                       | chronic_heat/osc/v2/days_tas_above_50c_CMCC-ESM2_{scenario}_{year}                | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/50c           | Days with average temperature above 50°C/CNRM-CM6-1                      | chronic_heat/osc/v2/days_tas_above_50c_CNRM-CM6-1_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/50c           | Days with average temperature above 50°C/MIROC6                          | chronic_heat/osc/v2/days_tas_above_50c_MIROC6_{scenario}_{year}                   | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/50c           | Days with average temperature above 50°C/MPI-ESM1-2-LR                   | chronic_heat/osc/v2/days_tas_above_50c_MPI-ESM1-2-LR_{scenario}_{year}            | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/50c           | Days with average temperature above 50°C/NorESM2-MM                      | chronic_heat/osc/v2/days_tas_above_50c_NorESM2-MM_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/55c           | Days with average temperature above 55°C/ACCESS-CM2                      | chronic_heat/osc/v2/days_tas_above_55c_ACCESS-CM2_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/55c           | Days with average temperature above 55°C/CMCC-ESM2                       | chronic_heat/osc/v2/days_tas_above_55c_CMCC-ESM2_{scenario}_{year}                | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/55c           | Days with average temperature above 55°C/CNRM-CM6-1                      | chronic_heat/osc/v2/days_tas_above_55c_CNRM-CM6-1_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/55c           | Days with average temperature above 55°C/MIROC6                          | chronic_heat/osc/v2/days_tas_above_55c_MIROC6_{scenario}_{year}                   | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/55c           | Days with average temperature above 55°C/MPI-ESM1-2-LR                   | chronic_heat/osc/v2/days_tas_above_55c_MPI-ESM1-2-LR_{scenario}_{year}            | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_tas/above/55c           | Days with average temperature above 55°C/NorESM2-MM                      | chronic_heat/osc/v2/days_tas_above_55c_NorESM2-MM_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | days_wbgt_above              | Days with wet-bulb globe temperature above threshold in °C/ACCESS-CM2    | chronic_heat/osc/v2/days_wbgt_above_ACCESS-CM2_{scenario}_{year}                  | historical (2005), SSP126 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP245 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP370 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP585 (2030, 2040, 2050, 2060, 2070, 2080, 2090)                             |
| ChronicHeat        | days_wbgt_above              | Days with wet-bulb globe temperature above threshold in °C/CMCC-ESM2     | chronic_heat/osc/v2/days_wbgt_above_CMCC-ESM2_{scenario}_{year}                   | historical (2005), SSP126 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP245 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP370 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP585 (2030, 2040, 2050, 2060, 2070, 2080, 2090)                             |
| ChronicHeat        | days_wbgt_above              | Days with wet-bulb globe temperature above threshold in °C/CNRM-CM6-1    | chronic_heat/osc/v2/days_wbgt_above_CNRM-CM6-1_{scenario}_{year}                  | historical (2005), SSP126 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP245 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP370 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP585 (2030, 2040, 2050, 2060, 2070, 2080, 2090)                             |
| ChronicHeat        | days_wbgt_above              | Days with wet-bulb globe temperature above threshold in °C/MIROC6        | chronic_heat/osc/v2/days_wbgt_above_MIROC6_{scenario}_{year}                      | historical (2005), SSP126 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP245 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP370 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP585 (2030, 2040, 2050, 2060, 2070, 2080, 2090)                             |
| ChronicHeat        | days_wbgt_above              | Days with wet-bulb globe temperature above threshold in °C/MPI-ESM1-2-LR | chronic_heat/osc/v2/days_wbgt_above_MPI-ESM1-2-LR_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP245 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP370 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP585 (2030, 2040, 2050, 2060, 2070, 2080, 2090)                             |
| ChronicHeat        | days_wbgt_above              | Days with wet-bulb globe temperature above threshold in °C/NorESM2-MM    | chronic_heat/osc/v2/days_wbgt_above_NorESM2-MM_{scenario}_{year}                  | historical (2005), SSP126 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP245 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP370 (2030, 2040, 2050, 2060, 2070, 2080, 2090), SSP585 (2030, 2040, 2050, 2060, 2070, 2080, 2090)                             |
| ChronicHeat        | mean_degree_days/above/32c   | Mean degree days above 32°C/ACCESS-CM2                                   | chronic_heat/osc/v2/mean_degree_days_v2_above_32c_ACCESS-CM2_{scenario}_{year}    | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/above/32c   | Mean degree days above 32°C/CMCC-ESM2                                    | chronic_heat/osc/v2/mean_degree_days_v2_above_32c_CMCC-ESM2_{scenario}_{year}     | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/above/32c   | Mean degree days above 32°C/CNRM-CM6-1                                   | chronic_heat/osc/v2/mean_degree_days_v2_above_32c_CNRM-CM6-1_{scenario}_{year}    | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/above/32c   | Mean degree days above 32°C/MIROC6                                       | chronic_heat/osc/v2/mean_degree_days_v2_above_32c_MIROC6_{scenario}_{year}        | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/above/32c   | Mean degree days above 32°C/MPI-ESM1-2-LR                                | chronic_heat/osc/v2/mean_degree_days_v2_above_32c_MPI-ESM1-2-LR_{scenario}_{year} | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/above/32c   | Mean degree days above 32°C/NorESM2-MM                                   | chronic_heat/osc/v2/mean_degree_days_v2_above_32c_NorESM2-MM_{scenario}_{year}    | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/above/index | Mean degree days above index value/ACCESS-CM2                            | chronic_heat/osc/v2/mean_degree_days_above_index_ACCESS-CM2_{scenario}_{year}     | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/above/index | Mean degree days above index value/CMCC-ESM2                             | chronic_heat/osc/v2/mean_degree_days_above_index_CMCC-ESM2_{scenario}_{year}      | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/above/index | Mean degree days above index value/CNRM-CM6-1                            | chronic_heat/osc/v2/mean_degree_days_above_index_CNRM-CM6-1_{scenario}_{year}     | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/above/index | Mean degree days above index value/MIROC6                                | chronic_heat/osc/v2/mean_degree_days_above_index_MIROC6_{scenario}_{year}         | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/above/index | Mean degree days above index value/MPI-ESM1-2-LR                         | chronic_heat/osc/v2/mean_degree_days_above_index_MPI-ESM1-2-LR_{scenario}_{year}  | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/above/index | Mean degree days above index value/NorESM2-MM                            | chronic_heat/osc/v2/mean_degree_days_above_index_NorESM2-MM_{scenario}_{year}     | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/below/index | Mean degree days below index value/ACCESS-CM2                            | chronic_heat/osc/v2/mean_degree_days_below_index_ACCESS-CM2_{scenario}_{year}     | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/below/index | Mean degree days below index value/CMCC-ESM2                             | chronic_heat/osc/v2/mean_degree_days_below_index_CMCC-ESM2_{scenario}_{year}      | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/below/index | Mean degree days below index value/CNRM-CM6-1                            | chronic_heat/osc/v2/mean_degree_days_below_index_CNRM-CM6-1_{scenario}_{year}     | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/below/index | Mean degree days below index value/MIROC6                                | chronic_heat/osc/v2/mean_degree_days_below_index_MIROC6_{scenario}_{year}         | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/below/index | Mean degree days below index value/MPI-ESM1-2-LR                         | chronic_heat/osc/v2/mean_degree_days_below_index_MPI-ESM1-2-LR_{scenario}_{year}  | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_degree_days/below/index | Mean degree days below index value/NorESM2-MM                            | chronic_heat/osc/v2/mean_degree_days_below_index_NorESM2-MM_{scenario}_{year}     | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/high          | Mean work loss, high intensity/ACCESS-CM2                                | chronic_heat/osc/v2/mean_work_loss_high_ACCESS-CM2_{scenario}_{year}              | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/high          | Mean work loss, high intensity/CMCC-ESM2                                 | chronic_heat/osc/v2/mean_work_loss_high_CMCC-ESM2_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/high          | Mean work loss, high intensity/CNRM-CM6-1                                | chronic_heat/osc/v2/mean_work_loss_high_CNRM-CM6-1_{scenario}_{year}              | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/high          | Mean work loss, high intensity/MIROC6                                    | chronic_heat/osc/v2/mean_work_loss_high_MIROC6_{scenario}_{year}                  | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/high          | Mean work loss, high intensity/MPI-ESM1-2-LR                             | chronic_heat/osc/v2/mean_work_loss_high_MPI-ESM1-2-LR_{scenario}_{year}           | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/high          | Mean work loss, high intensity/NorESM2-MM                                | chronic_heat/osc/v2/mean_work_loss_high_NorESM2-MM_{scenario}_{year}              | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/low           | Mean work loss, low intensity/ACCESS-CM2                                 | chronic_heat/osc/v2/mean_work_loss_low_ACCESS-CM2_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/low           | Mean work loss, low intensity/CMCC-ESM2                                  | chronic_heat/osc/v2/mean_work_loss_low_CMCC-ESM2_{scenario}_{year}                | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/low           | Mean work loss, low intensity/CNRM-CM6-1                                 | chronic_heat/osc/v2/mean_work_loss_low_CNRM-CM6-1_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/low           | Mean work loss, low intensity/MIROC6                                     | chronic_heat/osc/v2/mean_work_loss_low_MIROC6_{scenario}_{year}                   | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/low           | Mean work loss, low intensity/MPI-ESM1-2-LR                              | chronic_heat/osc/v2/mean_work_loss_low_MPI-ESM1-2-LR_{scenario}_{year}            | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/low           | Mean work loss, low intensity/NorESM2-MM                                 | chronic_heat/osc/v2/mean_work_loss_low_NorESM2-MM_{scenario}_{year}               | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/medium        | Mean work loss, medium intensity/ACCESS-CM2                              | chronic_heat/osc/v2/mean_work_loss_medium_ACCESS-CM2_{scenario}_{year}            | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/medium        | Mean work loss, medium intensity/CMCC-ESM2                               | chronic_heat/osc/v2/mean_work_loss_medium_CMCC-ESM2_{scenario}_{year}             | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/medium        | Mean work loss, medium intensity/CNRM-CM6-1                              | chronic_heat/osc/v2/mean_work_loss_medium_CNRM-CM6-1_{scenario}_{year}            | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/medium        | Mean work loss, medium intensity/MIROC6                                  | chronic_heat/osc/v2/mean_work_loss_medium_MIROC6_{scenario}_{year}                | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/medium        | Mean work loss, medium intensity/MPI-ESM1-2-LR                           | chronic_heat/osc/v2/mean_work_loss_medium_MPI-ESM1-2-LR_{scenario}_{year}         | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | mean_work_loss/medium        | Mean work loss, medium intensity/NorESM2-MM                              | chronic_heat/osc/v2/mean_work_loss_medium_NorESM2-MM_{scenario}_{year}            | historical (2005), SSP126 (2030, 2040, 2050), SSP245 (2030, 2040, 2050), SSP585 (2030, 2040, 2050)                                                                                                                                                        |
| ChronicHeat        | weeks_water_temp_above       | Weeks with average water temperature above threshold in °C/E2O           | chronic_heat/nluu/v2/weeks_water_temp_above_E2O_{scenario}_{year}                 | historical (1991)                                                                                                                                                                                                                                         |
| ChronicHeat        | weeks_water_temp_above       | Weeks with average water temperature above threshold in °C/GFDL          | chronic_heat/nluu/v2/weeks_water_temp_above_GFDL_{scenario}_{year}                | historical (1991), RCP 2.6 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 4.5 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 6.0 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 8.5 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090) |
| ChronicHeat        | weeks_water_temp_above       | Weeks with average water temperature above threshold in °C/HadGEM        | chronic_heat/nluu/v2/weeks_water_temp_above_HadGEM_{scenario}_{year}              | historical (1991), RCP 2.6 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 4.5 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 6.0 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 8.5 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090) |
| ChronicHeat        | weeks_water_temp_above       | Weeks with average water temperature above threshold in °C/IPSL          | chronic_heat/nluu/v2/weeks_water_temp_above_IPSL_{scenario}_{year}                | historical (1991), RCP 2.6 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 4.5 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 6.0 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 8.5 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090) |
| ChronicHeat        | weeks_water_temp_above       | Weeks with average water temperature above threshold in °C/MIROC         | chronic_heat/nluu/v2/weeks_water_temp_above_MIROC_{scenario}_{year}               | historical (1991), RCP 2.6 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 4.5 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 6.0 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 8.5 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090) |
| ChronicHeat        | weeks_water_temp_above       | Weeks with average water temperature above threshold in °C/NorESM        | chronic_heat/nluu/v2/weeks_water_temp_above_NorESM_{scenario}_{year}              | historical (1991), RCP 2.6 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 4.5 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 6.0 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090), RCP 8.5 (2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090) |
| CoastalInundation  | flood_depth                  | Flood depth/5%, with subsidence (WRI)                                    | inundation/wri/v2/inuncoast_{scenario}_wtsub_{year}_0_perc_05                     | RCP 4.5 (2030, 2050, 2080), RCP 8.5 (2030, 2050, 2080)                                                                                                                                                                                                    |
| CoastalInundation  | flood_depth                  | Flood depth/50%, no subsidence (WRI)                                     | inundation/wri/v2/inuncoast_{scenario}_nosub_{year}_0_perc_50                     | RCP 4.5 (2030, 2050, 2080), RCP 8.5 (2030, 2050, 2080)                                                                                                                                                                                                    |
| CoastalInundation  | flood_depth                  | Flood depth/50%, with subsidence (WRI)                                   | inundation/wri/v2/inuncoast_{scenario}_wtsub_{year}_0_perc_50                     | RCP 4.5 (2030, 2050, 2080), RCP 8.5 (2030, 2050, 2080)                                                                                                                                                                                                    |
| CoastalInundation  | flood_depth                  | Flood depth/95%, no subsidence (WRI)                                     | inundation/wri/v2/inuncoast_{scenario}_nosub_{year}_0                             | RCP 4.5 (2030, 2050, 2080), RCP 8.5 (2030, 2050, 2080)                                                                                                                                                                                                    |
| CoastalInundation  | flood_depth                  | Flood depth/95%, with subsidence (WRI)                                   | inundation/wri/v2/inuncoast_{scenario}_wtsub_{year}_0                             | RCP 4.5 (2030, 2050, 2080), RCP 8.5 (2030, 2050, 2080)                                                                                                                                                                                                    |
| CoastalInundation  | flood_depth                  | Flood depth/baseline, no subsidence (WRI)                                | inundation/wri/v2/inuncoast_historical_nosub_hist_0                               | historical (1980)                                                                                                                                                                                                                                         |
| CoastalInundation  | flood_depth                  | Flood depth/baseline, with subsidence (WRI)                              | inundation/wri/v2/inuncoast_historical_wtsub_hist_0                               | historical (1980)                                                                                                                                                                                                                                         |
| CoastalInundation  | flood_depth/nosub/5          | Flood depth/5%, no subsidence (WRI)                                      | inundation/wri/v2/inuncoast_{scenario}_nosub_{year}_0_perc_05                     | RCP 4.5 (2030, 2050, 2080), RCP 8.5 (2030, 2050, 2080)                                                                                                                                                                                                    |
| CombinedInundation | flooded_fraction             | Flooded fraction (Jupiter)                                               | combined_flood/jupiter/v1/fraction_{scenario}_{year}                              | SSP126 (2020, 2030, 2040, 2050, 2075, 2100), SSP585 (2020, 2030, 2040, 2050, 2075, 2100)                                                                                                                                                                  |
| Drought            | months/spei12m/below/index   | Drought SPEI index                                                       | drought/osc/v1/months_spei12m_below_index_MIROC6_{scenario}_{year}                | SSP585 (2005, 2030, 2040, 2050, 2080)                                                                                                                                                                                                                     |
| Drought            | months/spei3m/below/-2       | Drought (Jupiter)                                                        | drought/jupiter/v1/months_spei3m_below_-2_{scenario}_{year}                       | SSP126 (2020, 2030, 2040, 2050, 2075, 2100), SSP585 (2020, 2030, 2040, 2050, 2075, 2100)                                                                                                                                                                  |
| Fire               | fire_probability             | Fire probability (Jupiter)                                               | fire/jupiter/v1/fire_probability_{scenario}_{year}                                | SSP126 (2020, 2030, 2040, 2050, 2075, 2100), SSP585 (2020, 2030, 2040, 2050, 2075, 2100)                                                                                                                                                                  |
| Hail               | days/above/5cm               | Large hail days per year (Jupiter)                                       | hail/jupiter/v1/days_above_5cm_{scenario}_{year}                                  | SSP126 (2020, 2030, 2040, 2050, 2075, 2100), SSP585 (2020, 2030, 2040, 2050, 2075, 2100)                                                                                                                                                                  |
| Precipitation      | max/daily/water_equivalent   | Precipitation (Jupiter)                                                  | precipitation/jupiter/v1/max_daily_water_equivalent_{scenario}_{year}             | SSP126 (2020, 2030, 2040, 2050, 2075, 2100), SSP585 (2020, 2030, 2040, 2050, 2075, 2100)                                                                                                                                                                  |
| RiverineInundation | flood_depth                  | Flood depth (TUDelft)                                                    | inundation/river_tudelft/v2/flood_depth_unprot_{scenario}_{year}                  | historical (1985), RCP 4.5 (2035, 2085), RCP 8.5 (2035, 2085)                                                                                                                                                                                             |
| RiverineInundation | flood_depth                  | Flood depth/GFDL-ESM2M (WRI)                                             | inundation/wri/v2/inunriver_{scenario}_0000GFDL-ESM2M_{year}                      | RCP 4.5 (2030, 2050, 2080), RCP 8.5 (2030, 2050, 2080)                                                                                                                                                                                                    |
| RiverineInundation | flood_depth                  | Flood depth/HadGEM2-ES (WRI)                                             | inundation/wri/v2/inunriver_{scenario}_0000HadGEM2-ES_{year}                      | RCP 4.5 (2030, 2050, 2080), RCP 8.5 (2030, 2050, 2080)                                                                                                                                                                                                    |
| RiverineInundation | flood_depth                  | Flood depth/IPSL-CM5A-LR (WRI)                                           | inundation/wri/v2/inunriver_{scenario}_00IPSL-CM5A-LR_{year}                      | RCP 4.5 (2030, 2050, 2080), RCP 8.5 (2030, 2050, 2080)                                                                                                                                                                                                    |
| RiverineInundation | flood_depth                  | Flood depth/MIROC-ESM-CHEM (WRI)                                         | inundation/wri/v2/inunriver_{scenario}_MIROC-ESM-CHEM_{year}                      | RCP 4.5 (2030, 2050, 2080), RCP 8.5 (2030, 2050, 2080)                                                                                                                                                                                                    |
| RiverineInundation | flood_depth                  | Flood depth/NorESM1-M (WRI)                                              | inundation/wri/v2/inunriver_{scenario}_00000NorESM1-M_{year}                      | RCP 4.5 (2030, 2050, 2080), RCP 8.5 (2030, 2050, 2080)                                                                                                                                                                                                    |
| RiverineInundation | flood_depth                  | Flood depth/baseline (WRI)                                               | inundation/wri/v2/inunriver_{scenario}_000000000WATCH_{year}                      | historical (1980)                                                                                                                                                                                                                                         |
| RiverineInundation | flood_sop                    | Standard of protection (TUDelft)                                         | inundation/river_tudelft/v2/flood_sop_{scenario}_{year}                           | historical (1985), RCP 4.5 (2035, 2085), RCP 8.5 (2035, 2085)                                                                                                                                                                                             |
| Subsidence         | land_subsidence_rate         | Land subsidence rate (Davydzenka et Al (2024))                           | subsidence/csm/v1/land_subsidence_rate_{scenario}_{year}                          | historical (2021)                                                                                                                                                                                                                                         |
| WaterRisk          | water_demand                 | Water demand in centimeters/year (Aqueduct 4.0)                          | water_risk/wri/v2/water_demand_{scenario}_{year}                                  | historical (1999), SSP126 (2030, 2050, 2080), SSP370 (2030, 2050, 2080), SSP585 (2030, 2050, 2080)                                                                                                                                                        |
| WaterRisk          | water_depletion              | Water depletion (Aqueduct 4.0)                                           | water_risk/wri/v2/water_depletion_{scenario}_{year}                               | historical (1999), SSP126 (2030, 2050, 2080), SSP370 (2030, 2050, 2080), SSP585 (2030, 2050, 2080)                                                                                                                                                        |
| WaterRisk          | water_depletion_category     | Water depletion category (Aqueduct 4.0)                                  | water_risk/wri/v2/water_depletion_category_{scenario}_{year}                      | historical (1999), SSP126 (2030, 2050, 2080), SSP370 (2030, 2050, 2080), SSP585 (2030, 2050, 2080)                                                                                                                                                        |
| WaterRisk          | water_stress                 | Water stress (Aqueduct 4.0)                                              | water_risk/wri/v2/water_stress_{scenario}_{year}                                  | historical (1999), SSP126 (2030, 2050, 2080), SSP370 (2030, 2050, 2080), SSP585 (2030, 2050, 2080)                                                                                                                                                        |
| WaterRisk          | water_stress_category        | Water stress category (Aqueduct 4.0)                                     | water_risk/wri/v2/water_stress_category_{scenario}_{year}                         | historical (1999), SSP126 (2030, 2050, 2080), SSP370 (2030, 2050, 2080), SSP585 (2030, 2050, 2080)                                                                                                                                                        |
| WaterRisk          | water_supply                 | Water supply in centimeters/year (Aqueduct 4.0)                          | water_risk/wri/v2/water_supply_{scenario}_{year}                                  | historical (1999), SSP126 (2030, 2050, 2080), SSP370 (2030, 2050, 2080), SSP585 (2030, 2050, 2080)                                                                                                                                                        |
| Wind               | max_speed                    | Max 1 minute sustained wind speed (Jupiter)                              | wind/jupiter/v1/max_1min_{scenario}_{year}                                        | SSP126 (2020, 2030, 2040, 2050, 2075, 2100), SSP585 (2020, 2030, 2040, 2050, 2075, 2100)                                                                                                                                                                  |
| Wind               | max_speed                    | Max wind speed (IRIS)                                                    | wind/iris/v1/max_speed_{scenario}_{year}                                          | historical (2010), SSP119 (2050), SSP245 (2050), SSP585 (2050)                                                                                                                                                                                            |

#### Requesting hazard indicators through physrisk directly

As mentioned above, the API is simply using a hosted version of physrisk and it is possible to run the same calculations using physrisk directly, as long as the necessary API keys are present in a credentials.env file.

The requests above can be run in physrisk using a Requester object.

In [7]:
load_dotenv("../../credentials.env")
# the container is a dependency injection container,
# which allows the calculation to be configured to a particular use-case
container = Container()
# the requester is used to run calculations using the API.
# At this point, we can of course debug into the code and modify as required.
requester = container.requester()
result = requester.get(request_id="get_hazard_data", request_dict=request)